In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [2]:
# Store Part I results into DataFrame
city_df = pd.read_csv("../python-api-challenge/cities.csv")
city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [3]:
# Store 'Lat' and 'Lng' into  locations
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
my_city_df= city_df.loc[(city_df["Humidity"] <50) & (city_df["Max Temp"] > 60) & 
                        (city_df["Wind Speed"] <10)& (city_df["Cloudiness"] ==0)].dropna()
my_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
340,340,la libertad,0,PE,1558378855,36,-16.38,-71.56,64.40,8.05
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
371,371,carballo,0,ES,1558378860,44,43.21,-8.69,63.00,5.82
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72


In [6]:
#Hotel Map 
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

hotel_df=my_city_df[["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]=""
hotel_df



,City,Country,Lat,Lng,Hotel Name
26,colonelganj,IN,27.13,81.70,
142,sarai mir,IN,26.02,82.92,
176,nador,MA,35.17,-2.93,
259,hami,CN,42.84,93.51,
340,la libertad,PE,-16.38,-71.56,
346,chiredzi,ZW,-21.05,31.67,
371,carballo,ES,43.21,-8.69,
416,kirkuk,IQ,35.47,44.40,
444,birjand,IR,32.86,59.22,
518,bonito,BR,-21.13,-56.49,


In [7]:
#Store the first Hotel result into the DataFrame.
params = {
    "radius": 5000,  
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat=row["Lat"]
    lng=row["Lng"]

    # add keyword to params dict
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # assemble url and make API request
    respones = requests.get(base_url, params=params).json()
    
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = respones["results"][0]["name"]
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,colonelganj,IN,27.13,81.70,जय मां दुर्गा मंदिर
142,sarai mir,IN,26.02,82.92,Rasoolpur Barwa
176,nador,MA,35.17,-2.93,Hôtel La Giralda
259,hami,CN,42.84,93.51,Post And Telecommunication Hotel
340,la libertad,PE,-16.38,-71.56,Hotel La cuesta de Cayma
346,chiredzi,ZW,-21.05,31.67,The Nesbitt Arms
371,carballo,ES,43.21,-8.69,Pazo do Souto
416,kirkuk,IQ,35.47,44.40,Bayaz Han (Beyaz Han)
444,birjand,IR,32.86,59.22,تاکسی سرویس آرامش
518,bonito,BR,-21.13,-56.49,Marruá Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))